<a href="https://colab.research.google.com/github/r0bert8841/Fantasy-Hockey-Predictions/blob/main/Dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<style>div.container { width: 100% }</style>
<img style="float:left;  vertical-align:text-bottom;" height="65" width="172" src="https://raw.githubusercontent.com/holoviz/holoviz/master/doc/_static/holoviz-logo-unstacked.svg" />
<div style="float:right; vertical-align:text-bottom;"><h2>Tutorial 4. Making dashboards</h2></div>


This tutorial demonstrates the Easiest Way to Create an Interactive Dashboard in Python from any DataFrame.  If you already know some Pandas, you can almost immediately use `hvPlot` `.interactive` and `Panel` to turn your DataFrame processing pipeline into a dashboard! It just takes a few lines of familiar code to make an interactive dashboard like this:
![Data App](https://github.com/holoviz-community/HoloViz_KDD2022/blob/main/assets/easy-dataframe-dashboards.gif?raw=1)



## Import and configure packages

Please note that in **Colab** you will need to `!pip install panel hvplot`.

In [ ]:
def environment():
    try:
        get_ipython()
        print('notebook')
        return str(get_ipython())
    except:
        print('server')
        return 'server'
env = environment()

if env=="server":
   theme="fast"
else:
   theme="simple"

if 'google.colab' in env:
    print('Running on CoLab')
    !pip install panel hvplot

notebook
Running on CoLab


In [ ]:
import panel as pn

pn.extension('tabulator', sizing_mode="stretch_width")

In [ ]:
import hvplot.pandas
import holoviews as hv

hv.extension('bokeh')

## Load Data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/hockey_stats/features.csv', sep='|')

In [ ]:
df.columns

Index(['Unnamed: 0', 'season', 'player_id', 'date', 'team_id', 'game_count',
       'game_count_curr_season', 'game_count_team', 'min_season',
       'min_season_team', 'max_games', 'goals_mean_career', 'goals_mean_82',
       'goals_team_mean_career', 'goals_team_mean_82', 'goals_team_mean_41',
       'goals_mean_career_max', 'goals_mean_82_max',
       'games_since_goals_mean_career_max', 'games_since_goals_mean_82_max',
       'pred_goals_mean_82', 'pred_goals_mean_82_abs_diff',
       'pred_goals_mean_82_new_max', 'pred_goals_mean_82_increase',
       'goals_mean_career_var', 'goals_mean_82_var', 'goals_mean_diff',
       'goals_mean_diff_perc', 'goals_mean_diff_perc_rolling_min',
       'goals_mean_diff_perc_rolling_max', 'goals_game_count_min',
       'goals_game_count_max', 'goals_slope_since_min',
       'goals_slope_since_max', 'goals_shift_41_games', 'goals_slope_41_games',
       'assists_mean_career', 'assists_mean_82', 'assists_team_mean_career',
       'assists_team_mean_

## Define DataFrame Pipeline

## Make DataFrame Pipeline Interactive

What if we would like to turn the values of cyl, the values of mfr, and the variable hp into interactive widgets that we can change and control? Is it possible? Yes, it is, and with hvPlot it’s not even difficult. Here are the steps:

- First, we need to wrap our dataframe with `.interactive()`: `idf = df.interactive()`, so that this dataframe becomes interactive and we can use Panel widgets on this dataframe.

“.interactive stores a copy of your pipeline (series of method calls or other expressions on your data) and dynamically replays the pipeline whenever that widget changes.”


In [ ]:
idf = df.interactive()

Define [Panel widgets](https://panel.holoviz.org/reference/index.html#widgets)

- Second, we can define the panel widgets we would like to use. Here I defined a panel widget for cylinders, a widget for the manufacturer, and a widget to select the y axis.


In [ ]:
cylinders = pn.widgets.IntSlider(name='Cylinders', start=4, end=8, step=2)
cylinders

In [ ]:
mfr = pn.widgets.ToggleGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    button_type='success')
mfr

In [ ]:
yaxis = pn.widgets.RadioButtonGroup(
    name='Y axis',
    options=['goals', 'assists','points'],
    button_type='success'
)
yaxis

RadioButtonGroup(button_type='success', name='Y axis', options=['goals', 'assists', ...], sizing_mode='stretch_width', value='goals')

In [ ]:
# prompt: can you create a dropdown box that lists all the fullNames and we can choose one

full_names = df['fullName'].unique().tolist() # Convert the NumPy array to a list
full_names.sort() # Sort the list alphabetically
dropdown = pn.widgets.Select(options=full_names, name='Full Name')
dropdown

Select(name='Full Name', options=['Aaron Ekblad', ...], sizing_mode='stretch_width', value='Aaron Ekblad')

- Finally, we can replace the values or variables from the original Pandas pipeline to these widgets we just defined. We define the output of the pipeline as ipipeline:


In [ ]:
ipipeline = (
    idf[
        (idf.fullName == dropdown)
    ][['season', 'player_id', 'date', 'team_id', 'game_count',
       'game_count_curr_season', 'game_count_team', 'min_season',
       'min_season_team', 'max_games', yaxis.value+'_mean_career', yaxis.value+'_mean_82',
       yaxis.value+'_team_mean_career', yaxis.value+'_team_mean_82', yaxis.value+'_team_mean_41',
       yaxis.value+'_mean_career_max', yaxis.value+'_mean_82_max',
       'games_since_'+yaxis.value+'_mean_career_max', 'games_since_'+yaxis.value+'_mean_82_max',
       yaxis.value+'_mean_career_var', yaxis.value+'_mean_82_var', yaxis.value+'_mean_diff',
       yaxis.value+'_mean_diff_perc', yaxis.value+'_mean_diff_perc_rolling_min',
       yaxis.value+'_mean_diff_perc_rolling_max', yaxis.value+'_game_count_min',
       yaxis.value+'_game_count_max', yaxis.value+'_slope_since_min',
       yaxis.value+'_slope_since_max', yaxis.value+'_shift_41_games', yaxis.value+'_slope_41_games',
       'pred_'+yaxis.value+'_mean_82', 'pred_'+yaxis.value+'_mean_82_abs_diff',
       'pred_'+yaxis.value+'_mean_82_new_max', 'pred_'+yaxis.value+'_mean_82_increase']]

    .sort_values(by='game_count', ascending=False)
    .reset_index(drop=True)
)
ipipeline.head()

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

## Pipe to Table

In [ ]:
itable = ipipeline.pipe(pn.widgets.Tabulator, pagination='remote', page_size=10, theme=theme)
itable

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

Check out the [Tabulator Reference Guide](https://panel.holoviz.org/reference/widgets/Tabulator.html) for more inspiration.

## Pipe to hvplot

In [ ]:
PALETTE = ["#ff6f69", "#ffcc5c", "#88d8b0", ]
pn.Row(
    pn.layout.HSpacer(height=50, background=PALETTE[0]),
    pn.layout.HSpacer(height=50, background=PALETTE[1]),
    pn.layout.HSpacer(height=50, background=PALETTE[2]),
)

<ipython-input-38-0bed645b6b2f>:3: PanelDeprecationWarning: "HSpacer(..., background='#ff6f69')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ff6f69'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[0]),
<ipython-input-38-0bed645b6b2f>:4: PanelDeprecationWarning: "HSpacer(..., background='#ffcc5c')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#ffcc5c'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[1]),
<ipython-input-38-0bed645b6b2f>:5: PanelDeprecationWarning: "HSpacer(..., background='#88d8b0')" is deprecated and will be removed in version 1.4, use "HSpacer(..., styles={'background': '#88d8b0'})" instead.
  pn.layout.HSpacer(height=50, background=PALETTE[2]),


Row(sizing_mode='stretch_width')
    [0] HSpacer(background='#ff6f69', height=50, styles={'background': '#ff6f69'})
    [1] HSpacer(background='#ffcc5c', height=50, styles={'background': '#ffcc5c'})
    [2] HSpacer(background='#88d8b0', height=50, styles={'background': '#88d8b0'})

In [ ]:
ihvplot = ipipeline.hvplot(x='game_count', y=yaxis.value+'_mean_diff_perc', color=PALETTE, line_width=6, height=400)
ihvplot

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

## Layout using Panel

In [ ]:
pn.Column(
    pn.Row(
       dropdown, yaxis
    ),
    ihvplot.panel(),
    itable.panel()
)

Column(sizing_mode='stretch_width')
    [0] Row(sizing_mode='stretch_width')
        [0] Select(name='Full Name', options=['Aaron Ekblad', ...], sizing_mode='stretch_width', value='Ivan Provorov')
        [1] RadioButtonGroup(button_type='success', name='Y axis', options=['goals', 'assists', ...], sizing_mode='stretch_width', value='points')
    [1] ParamFunction(function, _pane=HoloViews, defer_load=False, sizing_mode='stretch_width')
    [2] ParamFunction(function, _pane=Tabulator, defer_load=False, sizing_mode='stretch_width')

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-packages/holoviews/core/util.py:1585: PanelDeprecationWarning: 'param_value_if_widget' is deprecated and will be removed in version 1.4, use 'transform_reference' instead.
  value = param_value_if_widget(value)
/usr/local/lib/python3.10/dist-p

## Layout using Template

Here we use the [FastListTemplate](https://panel.holoviz.org/reference/templates/FastListTemplate.html#templates-gallery-fastlisttemplate).

In [ ]:
template = pn.template.FastListTemplate(
    title='Interactive DataFrame Dashboards with hvplot .interactive',
    sidebar=['Players', dropdown, 'Y axis' , yaxis],
    main=[ihvplot.panel(), itable.panel()],
    accent_base_color="#88d8b0",
    header_background="#88d8b0",
)
# template.show()
template.servable();

Please note that to get the Tabulator table styled nicely for dark mode you can set `theme='fast'` when you define the `itable`. It won't work in the notebook though.

To *serve the notebook* run `panel serve 04_Dashboard.ipynb`.

In [ ]:
template.show()

Launching server at http://localhost:45941


<div class="alert alert-info">
<h4>Exercise</h4>
    
Take a look at the [Panel widget docs](https://panel.holoviz.org/reference/index.html#widgets). And change the three widgets above to different types of widgets of your choice.  
        
<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python

cylinders = pn.widgets.IntInput(name='Cylinders', value=4, step=2, start=4, end=8)
mfr = pn.widgets.CheckBoxGroup(
    name='MFR',
    options=['ford', 'chevrolet', 'honda', 'toyota', 'audi'],
    value=['ford', 'chevrolet', 'honda', 'toyota', 'audi']
)
yaxis = pn.widgets.RadioBoxGroup(
    name='Y axis',
    options=['hp', 'weight'],
)
    
    
```

</details>
</div>

<div class="alert alert-info">
<h4>Exercise</h4>
    
Add a widget to select origin in the example above.

<details><summary><i><u>(Solution)</u><i></summary><br>
    
```python

origin = pn.widgets.ToggleGroup(
    name='Origin',
    options=['North America', 'Asia', 'Europe'],
    value=['North America', 'Asia', 'Europe'],
    button_type='success')
origin
    
ipipeline = (
    idf[
        (idf.cyl == cylinders) &
        (idf.mfr.isin(mfr)) &
        (idf.origin.isin(origin))
    ]
    .groupby(['origin', 'mpg'])[yaxis].mean()
    .to_frame()
    .reset_index()
    .sort_values(by='mpg')  
    .reset_index(drop=True)
)
ipipeline.head()
    
```

</details>
</div>

<div class="alert alert-info">
<h4>Exercise</h4>
    
Try save this notebook to a Python file, or copy and paste the relevant code to a Python file.
    
Then serve the dashboard from the Python file: `panel serve 04_Dashboard.py`.

</details>
</div>

<div class="alert alert-info">
<h4>Reading time </h4>
    
Read the [Panel documentation](https://panel.holoviz.org/) and let us know if you have any questions.
</div>